In [1]:
import numpy as np
import os
import tensorflow as tf
n_inputs=3 # 입력데이터
n_neurons = 5 # 셀의 가중치 사이즈
tf.reset_default_graph() # 그라프 초기화(변수 생성)

X0 = tf.placeholder(tf.float32, [None, n_inputs]) # 4x3 말단데이터 개수가 3
X1 = tf.placeholder(tf.float32, [None, n_inputs]) # 4x3
#[0,1,2] 사이즈가 하나의 셀로 입력
# FFNN(feed forweard neural network)

# 가중치 사이즈 => 특성을 찾아내는  것
basic_cell = tf.contrib.rnn.BasicRNNCell(num_units=n_neurons)
# static_rnn : rnn network : 4개의 셀이 자동으로 연결된면서 메모리 확보
output_seqs,states = tf.contrib.rnn.static_rnn(basic_cell,[X0,X1],dtype=tf.float32) # 2x4x3 , 배치사이즈 * 셀 수 * 뉴런 수

Y0,Y1 = output_seqs

# 출력, 다음셀로 전달되는 값 : 마지막 states값(수평으로 셀을 연결)
init = tf.global_variables_initializer()
X0_batch = np.array([[0,1,2],[3,4,5],[6,7,8],[9,0,1]]) # 4x3
X1_batch = np.array([[9,8,7],[0,0,0],[6,5,4],[3,2,1]])

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
This class is equivalent as tf.keras.layers.SimpleRNNCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell, unroll=True)`, which is equivalent to this API
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [2]:
with tf.Session() as sess:
#     sess.run(init)
    init.run()
    Y0_val, Y1_val = sess.run([Y0,Y1], feed_dict={X0:X0_batch, X1:X1_batch})
    
    print('처음 데이터 특성 : \n{} \n차수 : {} '.format(Y0_val, Y0_val.shape))
    print('두번째 데이터 특성 : \n{} \n차수 : {} '.format(Y1_val, Y1_val.shape))
    # 4x3 => 4x5

처음 데이터 특성 : 
[[-0.33641216  0.3341674   0.30749053 -0.7726948   0.7692158 ]
 [-0.7707651   0.16922277  0.94614005 -0.9998492   0.922941  ]
 [-0.9347026  -0.00577932  0.9971119  -0.9999998   0.9756778 ]
 [ 0.1724769  -0.7828837   0.9998124  -0.9999937  -0.9973935 ]] 
차수 : (4, 5) 
두번째 데이터 특성 : 
[[-0.95321894 -0.8626379   0.999878   -1.          0.89323056]
 [-0.01320682 -0.620482    0.51450783 -0.52114797  0.489258  ]
 [-0.72890836 -0.8911757   0.99628913 -0.9999986   0.5845853 ]
 [-0.40252233 -0.8914003   0.45425022 -0.98890555 -0.9148378 ]] 
차수 : (4, 5) 


? 셀이 가지고 있는 가중치 사이즈 : 3x5?

## 분류

In [3]:
tf.reset_default_graph() # 그라프 초기화(변수 생성)
n_steps = 28 # 셀수 
n_inputs = 28 # 셀당 인풋사이즈
n_neurons = 150 # 뉴런
n_outputs = 10 # 확률사이즈
learning_rate = 0.001

# 3차원으로 데이터를 받아오기때문에
X = tf.placeholder(tf.float32, [None, n_steps, n_inputs]) # ? x 28x 28
y = tf.placeholder(tf.int32, [None])

# FFNN
basic_cell = tf.contrib.rnn.BasicRNNCell(num_units=n_neurons) #150
# 데이터가 3차원으로 입력됨
# 2차원이 셀수
# 셀 : 28개
# state 마지막셀의 수평으로 전달되는 값
# state 마지막셀의 output과 같다.
# 28개의 셀이 있는데 마지막 한개의 output을 사용 => many to one
# 감정분류 -> 분류
# state의 차수 : 150x150
# output의 차수 : latent time 지연시간을 통해 계산된 셀의 
# 모든 값을 결합 출력 150x28x150
outputs, states = tf.nn.dynamic_rnn(basic_cell, X,dtype=tf.float32)

# 150개의 특징중에 10개만 추출
# dense 입력차수, 출력차수만 지정하면 자동으로 바이어스를 생성
# 가중치 공간을 확보해서 
# 150x150, => 150x10
# dense.의 가중치 사이즈 = 150x10
logits = tf.layers.dense(states, n_outputs)
# 분류를 위한 미분이 가능한 식으로 바뀜
xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels = y, logits=logits)

loss = tf.reduce_mean(xentropy) # 배치사이즈  => 평균을 통해서 loss를 구한다.

# momentum _ propgrad
optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate)
training_op = optimizer.minimize(loss)

# 양쪽에서 큰놈으로 
# 가장큰값
correct = tf.nn.in_top_k(logits, y, 1)
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
init = tf.global_variables_initializer()

Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Use keras.layers.dense instead.


In [4]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('/tmp/data/')
# 원래이미지 모양으로 학습한다.
X_test = mnist.test.images.reshape((-1,n_steps, n_inputs))
y_test = mnist.test.labels

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [5]:
# n_epochs= 100
n_epochs= 10
batch_size = 150
with tf.Session() as sess:
    init.run()
    for epoch in  range(n_epochs):
        for iteration in range(mnist.train.num_examples // batch_size): # 60000/150 을 100번 반복해서 돈다.
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            # 이미지 사이즈로 생성
            X_batch = X_batch.reshape((-1,n_steps, n_inputs))
            sess.run(training_op, feed_dict={X:X_batch, y:y_batch})
        acc_train = accuracy.eval(feed_dict={X:X_batch, y:y_batch})
        acc_test = accuracy.eval(feed_dict={X:X_test, y:y_test})
        print(epoch, 'Train accuracy : ', acc_train, 'Test accyract : ',acc_test)

0 Train accuracy :  0.93333334 Test accyract :  0.922
1 Train accuracy :  0.97333336 Test accyract :  0.9456
2 Train accuracy :  0.96 Test accyract :  0.9505
3 Train accuracy :  0.98 Test accyract :  0.9561
4 Train accuracy :  0.9266667 Test accyract :  0.9649
5 Train accuracy :  0.9866667 Test accyract :  0.9545
6 Train accuracy :  0.97333336 Test accyract :  0.968
7 Train accuracy :  0.96666664 Test accyract :  0.96
8 Train accuracy :  0.96666664 Test accyract :  0.9732
9 Train accuracy :  0.9866667 Test accyract :  0.9716


In [6]:
#multi rnn cell
# 오타 찾아라

In [7]:
# # BasicRNNCell : 기본데이터가 입력
# # MultiRNNCell : 수직으로 레이어 구성 : 수직우로 셀이 구성
# # dynamic_rnn : 모델

# import tensorflow as tf
# tf.reset_default_graph() # 그라프 초기화(변수 생성)

# n_steps = 28
# n_inputs = 28
# n_outputs = 10
# learning_rate = 0.001
# X = tf.placeholder(tf.float32, [None, n_steps, n_inputs])
# y = tf.placeholder(tf.float32, [None])

# n_neurons = 100
# n_layers = 3 # 3개의 멀티 레이어 : 3개의 셀을 생성
# layers = [tf.contrib.rnn.BasicRNNCell(num_units=n_neurons, activation=tf.nn.relu) for layer in range(n_layers)]

# # 3개를 조합해서 MultiRNNCell을 만든다.
# multi_layer_cell = tf.contrib.rnn.MultiRNNCell(layers)

                                
                                

In [8]:
# # dynamic_rnn : 셀로 입력 되는 데이터의 개수에 맞추어서 셀을 구성
# # 나는 학교에 간다. ->  static_rnn 동일한 사이즈 : 큰것 기준, 작은것은 padding
# # 입력사이즈 변동 => 가중치 조절 => 나가는 특징은 일치(neural수가 같다.)
# # 3층 셀이 28개가 조성 : 고정 사이즈
# outputs, states = tf.nn.dynamic_rnn(multi_layer_cell, X, dtype=tf.float32)

# # states가 몇개가 발생하는가
# # 3x150x100
# states_concat = tf.concat(axis = 1, values=states) # 150x300
# logits = tf.layers.dense(states_concat, n_outputs) # 가중치 :  300x10 => 150x10 확률값
# # 원핫인코딩 * log(예측값)
# xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels = y, logits=logits)

# loss = rf.reduce.mean(xectropy)

# optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
# training_op = optimizer.minimize(loss)
# coreect =tf.nn.in_top_k(logits, y,1)
# accuracy = tf.reduce_mean(tf.cats(correct, tf.float32))
# init = tf.global_varibles_initializer()

In [9]:
# n_epochs= 100
# batch_size = 150
# with tf.Session() as sess:
#     init.run()
#     for epoch in  range(n_epochs):
#         for iteration in range(mnist.train.num_examples // batch_size): # 60000/150 을 100번 반복해서 돈다.
#             X_batch, y_batch = mnist.train.next_batch(batch_size)
#             # 이미지 사이즈로 생성
#             X_batch = X_batch.reshape((-1,n_steps, n_inputs))
#             sess.run(training_op, feed_dict={X:X_batch, y:y_batch})
#         acc_train = accuracy.eval(feed_dict={X:X_batch, y:y_batch})
#         acc_test = accuracy.eval(feed_dict={X:X_test, y:y_test})
#         print(epoch, 'Train accuracy : ', acc_train, 'Test accyract : ',acc_test)

In [10]:
#  MUltiRNNCELll 을 할것이다 >> 3단계로 구성되어있다  (basic rnncell, multi rnncell, dynamic_rnn 으로 구성)
#BasicRNNCell : 기본데이터가 입력되는 곳은 basicrnncell 이다!! 
# MultiRNNCell : 수직으로 레이어 구성( 아까는 이거 없었음 )  수평으로 구성되어있었는데 이번에는 수직으로도 구성 >> 밑에보면 3개로
#            >>> multirnncell 로 의해서 수직으로 셀이 구성 
# dynamic_rnn : 이것이 모델이다 
tf.reset_default_graph()  # 
n_steps= 28
n_inputs=28
n_outputs=10
learning_rate = 0.001
X = tf.placeholder(tf.float32, [None, n_steps, n_inputs])
y = tf.placeholder(tf.int32, [None])
n_neurons=100
n_layers=3  # 3개의 멀티레이어 
layers=[tf.contrib.rnn.BasicRNNCell(num_units=n_neurons, activation=tf.nn.relu) for layer in range(n_layers)]  # basicrnncell 은  3개만들어짐 for 문통해 확인가능>>> 3개의 셀 생성

multi_layer_cell = tf.contrib.rnn.MultiRNNCell(layers) # 3개의 셀을 조합해서 multirnncell 을 만든다 ( 셀이 3층으로 구성되어있다 ) 

Instructions for updating:
This class is equivalent as tf.keras.layers.StackedRNNCells, and will be replaced by that in Tensorflow 2.0.


In [11]:
# 이번엔 dynamic_rnn 이다  : 셀로입력되는 데이터의 개수에 맞추어서 셀을 구성할 수 있다 
#ex )  나는 학교에 간다. ( 2글자 + 3글자+2글자 --> 사이즈가 다르다 -> 
#  사이즈가 다르면 static_rnn 에서는  동일한사이즈로 맞췄다.-> 큰것을 기준으로 작은 것을 padding 했었다 >> 결과가 안좋게 나오기떄문에 dynamic_rnn 을 쓴다 
#  dynamic_rnn 은 사이즈가 다르면 다른대로 상관하지 않는다. (입력사이즈를 변동>>> 가중치를 조절한다 >> 나가는 특징은 일치함 = neural 수는 같다 ) 

# 3층 셀이 28개가 조성이 된다 :: 고정사이즈이다  
outputs, states = tf.nn.dynamic_rnn(multi_layer_cell, X, dtype=tf.float32)

# states 가 몇개가 발생하는가?? 
# 3x 150x 100  ( 층이 3층이므로 3층, 배치사이즈가 150이니까 150,   150x100 인데 3층이니까 3이붙는것이다 )

states_concat = tf.concat(axis=1, values= states)  # concat 이 열방향으로 하게됨 (axis=1 에 의해서) >>> 150x300 이 된다  
logits= tf.layers.dense(states_concat, n_outputs)  # dense : 입력차수와 출력차수를 넣어주면 가중치 사이즈를 자기가 만든다 
                                            #150x300 이 들어오고,  출력차수가 10이니까 가중치는 300 x10 이 됨   

    #가중치계산해서 나온결과는  150( 미니배치사이즈 ) x10 ( 확률적으로 나타나는 10개의 특징=확률값)
        # 원핫인코딩 x log( 에측값)  ? ? 
    
xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y,  logits=logits)
loss= tf.reduce_mean(xentropy)
optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate)
training_op = optimizer.minimize(loss)
correct = tf.nn.in_top_k(logits,y,1)
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
init = tf.global_variables_initializer()

In [12]:
n_epochs= 10
batch_size = 150
with tf.Session() as sess:
    init.run()
    for epoch in  range(n_epochs):
        for iteration in range(mnist.train.num_examples // batch_size): # 60000/150 을 100번 반복해서 돈다.
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            # 이미지 사이즈로 생성
            X_batch = X_batch.reshape((-1,n_steps, n_inputs))
            sess.run(training_op, feed_dict={X:X_batch, y:y_batch})
        acc_train = accuracy.eval(feed_dict={X:X_batch, y:y_batch})
        acc_test = accuracy.eval(feed_dict={X:X_test, y:y_test})
        print(epoch, 'Train accuracy : ', acc_train, 'Test accyract : ',acc_test)

0 Train accuracy :  0.9533333 Test accyract :  0.9469
1 Train accuracy :  0.9533333 Test accyract :  0.962
2 Train accuracy :  0.97333336 Test accyract :  0.9686
3 Train accuracy :  0.96 Test accyract :  0.969
4 Train accuracy :  0.98 Test accyract :  0.9779
5 Train accuracy :  0.99333334 Test accyract :  0.978
6 Train accuracy :  0.9866667 Test accyract :  0.9784
7 Train accuracy :  0.97333336 Test accyract :  0.9831
8 Train accuracy :  0.99333334 Test accyract :  0.9828
9 Train accuracy :  0.99333334 Test accyract :  0.9832


In [13]:
import tensorflow as tf
from tensorflow.python.framework import ops
ops.reset_default_graph()
from tensorflow.examples.tutorials.mnist import input_data
mnist=input_data.read_data_sets('./mnist/data/', one_hot=True)

learning_rate = 0.001
total_epoch = 10
batch_size=128
n_input=28
n_step=28
n_hidden=128
n_class=10

X=tf.placeholder(tf.float32, [None,n_step, n_input]) # 128x28x28
Y= tf.placeholder(tf.float32, [None,n_class]) # 128x10
W= tf.Variable(tf.random_normal([n_hidden, n_class])) # 128x10
b= tf.Variable(tf.random_normal([n_class])) # 10
# 28x128 사이즈의 가중치
cell = tf.nn.rnn_cell.BasicRNNCell(n_hidden) # neurons : 128:특징으로
print('cell',cell) # => 
# cell 수 : 28
# FFNN
outputs, states = tf.nn.dynamic_rnn(cell, X, dtype=tf.float32)
print('1. ',outputs) # =>? x 28x128
# outputs : 128x28x128, states : 128x128
outputs = tf.transpose(outputs, [1,0,2]) # 28x128x128
print('2. ',outputs) # => 28x?x128
# cell로 판단하겠다. 한장에 128개의 셀이 있다 그것이 28장이있다.
outputs = outputs[-1] # 맨마지막 셀: 128x128
print('3. ',outputs) # => ?x128
model = tf.matmul(outputs, W) + b # 128x128, 128x10, 10 => 128x10
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=model, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

Extracting ./mnist/data/train-images-idx3-ubyte.gz
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz
cell <tensorflow.python.ops.rnn_cell_impl.BasicRNNCell object at 0x000001718D9E5A88>
1.  Tensor("rnn/transpose_1:0", shape=(?, 28, 128), dtype=float32)
2.  Tensor("transpose:0", shape=(28, ?, 128), dtype=float32)
3.  Tensor("strided_slice:0", shape=(?, 128), dtype=float32)
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [14]:
sess= tf.Session()
sess.run(tf.global_variables_initializer())
# 60000/128
total_batch = int(mnist.train.num_examples/batch_size)
for epoch in range(total_epoch): # 10
    total_cost = 0
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        # 이미지 사이즈로 리세입
        batch_xs = batch_xs.reshape((batch_size, n_step, n_input))
        # _ : python이 계산한 마지막 결과가 저장되는것
        _, cost_val = sess.run([optimizer, cost], feed_dict={X:batch_xs, Y:batch_ys})
        
        total_cost += cost_val
    print("Epoch:", "%04d" % (epoch +1), 'Avg. cost =', '{:.3f}'.format(total_cost / total_batch))
print("최적화 완료!")

Epoch: 0001 Avg. cost = 0.562
Epoch: 0002 Avg. cost = 0.236
Epoch: 0003 Avg. cost = 0.179
Epoch: 0004 Avg. cost = 0.148
Epoch: 0005 Avg. cost = 0.140
Epoch: 0006 Avg. cost = 0.125
Epoch: 0007 Avg. cost = 0.111
Epoch: 0008 Avg. cost = 0.116
Epoch: 0009 Avg. cost = 0.099
Epoch: 0010 Avg. cost = 0.098
최적화 완료!


In [15]:
outputs

<tf.Tensor 'strided_slice:0' shape=(?, 128) dtype=float32>

In [16]:
# 문제 
# 1. 테스트 데이를 이용하여 테스트 하는 회로를 구성하시요
# 2. 3개의 셀을 갖는 multi_layer cell로 수정하시요

In [17]:
# 테스트?
sess= tf.Session()
sess.run(tf.global_variables_initializer())
# 60000/128
total_batch = int(mnist.test.num_examples/batch_size)
for epoch in range(total_epoch): # 10
    total_cost = 0
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.test.next_batch(batch_size)
        # 이미지 사이즈로 리세입
        batch_xs = batch_xs.reshape((batch_size, n_step, n_input))
        # _ : python이 계산한 마지막 결과가 저장되는것
        _, cost_val = sess.run([optimizer, cost], feed_dict={X:batch_xs, Y:batch_ys})
        
        total_cost += cost_val
    print("Epoch:", "%04d" % (epoch +1), 'Avg. cost =', '{:.3f}'.format(total_cost / total_batch))
print("최적화 완료!")

Epoch: 0001 Avg. cost = 1.103
Epoch: 0002 Avg. cost = 0.519
Epoch: 0003 Avg. cost = 0.395
Epoch: 0004 Avg. cost = 0.319
Epoch: 0005 Avg. cost = 0.273
Epoch: 0006 Avg. cost = 0.221
Epoch: 0007 Avg. cost = 0.211
Epoch: 0008 Avg. cost = 0.196
Epoch: 0009 Avg. cost = 0.155
Epoch: 0010 Avg. cost = 0.140
최적화 완료!


In [18]:
# 2. 3개의 셀을 갖는 multi_layer cell로 수정하시요

In [19]:
import tensorflow as tf
from tensorflow.python.framework import ops
ops.reset_default_graph()
from tensorflow.examples.tutorials.mnist import input_data
mnist=input_data.read_data_sets('./mnist/data/', one_hot=True)

learning_rate = 0.001
total_epoch = 10
batch_size=128
n_input=28
n_step=28
n_hidden=128
n_class=10
n_layers=3  # 3개의 멀티레이어 

X=tf.placeholder(tf.float32, [None,n_step, n_input]) # 128x28x28
Y= tf.placeholder(tf.float32, [None,n_class]) # 128x10
W= tf.Variable(tf.random_normal([n_hidden, n_class])) # 128x10
b= tf.Variable(tf.random_normal([n_class])) # 10

# 3개의 셀로 멀티레이어 생성해도 output은 하나의 셀일 경유와 동일
# state에서는 3개의 레이어마다 생성
layers=[tf.contrib.rnn.BasicRNNCell(num_units=n_hidden, activation=tf.nn.relu) for layer in range(n_layers)]  

multi_layer_cell = tf.contrib.rnn.MultiRNNCell(layers) 

Extracting ./mnist/data/train-images-idx3-ubyte.gz
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz


In [20]:
outputs, states = tf.nn.dynamic_rnn(multi_layer_cell, X, dtype=tf.float32)

outputs = tf.transpose(outputs, [1,0,2]) 

outputs = outputs[-1] 

model = tf.matmul(outputs, W) + b # 128x128, 128x10, 10 => 128x10
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=model, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

In [21]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
is_correct = tf.equal(tf.argmax(model,1), tf.argmax(Y,1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
test_batch_size = len(mnist.test.images)
test_xs = mnist.test.images.reshape(test_batch_size, n_step, n_input)
test_ys = mnist.test.labels
print('정확도 : ', sess.run(accuracy, feed_dict={X:test_xs, Y:test_ys}))

정확도 :  0.0833


## 규진언니

In [22]:
ops.reset_default_graph()

In [23]:
ops.reset_default_graph()
import tensorflow as tf
from tensorflow.python.framework import ops

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./mnist/data/", one_hot=True)
learning_rate = 0.001
total_epoch = 10
batch_size = 128
n_input = 28
n_step = 28
n_hidden = 128
n_class = 10
X = tf.placeholder(tf.float32, [None, n_step, n_input]) # 128X28X28
Y = tf.placeholder(tf.float32, [None, n_class]) # 128X10
W = tf.Variable(tf.random_normal([n_hidden, n_class])) #128X10
b = tf.Variable(tf.random_normal([n_class])) # 10

n_layers = 3
layers = [tf.contrib.rnn.BasicRNNCell(num_units=n_hidden, activation=tf.nn.relu) for layer in range(n_layers)]
multi_layer_cell = tf.contrib.rnn.MultiRNNCell(layers)
model = tf.matmul(outputs, W) + b
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=model, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)
outputs, states = tf.nn.dynamic_rnn(multi_layer_cell, X, dtype=tf.float32)
outputs = tf.transpose(outputs, [1,0,2])
outputs = outputs[-1]

is_correct = tf.equal(tf.argmax(model, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
test_batch_size = len(mnist.test.images)
test_xs = mnist.test.images.reshape(test_batch_size, n_step, n_input)
test_ys = mnist.test.labels
print('정확도: ', sess.run(accuracy, feed_dict={X: test_xs, Y: test_ys}))

Extracting ./mnist/data/train-images-idx3-ubyte.gz
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz


ValueError: Tensor("Variable:0", shape=(128, 10), dtype=float32_ref) must be from the same graph as Tensor("strided_slice:0", shape=(?, 128), dtype=float32).

In [ ]:
import numpy as np
values231 = np.array([[[1],[2],[3]],[[2],[3],[4]]])
print('입력데이터의 shape ', values231.shape) #2x3x1
import tensorflow as tf
tf.reset_default_graph()
tf_values231 = tf.constant(values231, dtype=tf.float32)

# 가중치 : 1x100 4개가 있어야한다.
lstm_cell = tf.contrib.rnn.LSTMCell(num_units = 100) # 뉴론 100

# rnn 모델은 동일 : 망과 망을 연결할 때 (번역망 : 한글, 영어)
# latent time 지연시간
outputs, state = tf.nn.dynamic_rnn(cell=lstm_cell, dtype=tf.float32, inputs=tf_values231)
# state : 2개가 나온다.
print(state.c.shape) # 2x100
print(state.h.shape) # 2x100
print('rnn이 출력하는outputs의 의미 ', outputs.shape) # 2x3x100 => 데이터 2개 셀은 3개
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    output_run, state_run = sess.run([outputs, state])
    print('output_run : ',output_run.shape)
    print('state_run.c : ',state_run.c.shape)
    print('state_run.h : ',state_run.h.shape)

In [ ]:
tf.reset_default_graph()
values = tf.constant(np.array([[[1],[2],[3]],[[2],[3],[4]]]), dtype=tf.float32)
lstm_cell_fw = tf.contrib.rnn.LSTMCell(100)
lstm_cell_bw = tf.contrib.rnn.LSTMCell(100)
# 양방향 LSTM
# Multi cell : 셀에서
(output_fw, output_bw), (output_state_fw, output_state_bw) = tf.nn.bidirectional_dynamic_rnn(cell_fw=lstm_cell_fw, cell_bw = lstm_cell_bw,inputs=values, dtype=tf.float32)

print(output_fw.shape)
print(output_bw.shape)


In [ ]:
# enbedding
import pprint
from tensorflow.contrib import rnn
from tensorflow.python.framework import ops
ops.reset_default_graph()
pp = pprint.PrettyPrinter(indent = 4) # 형태그대로?
sess = tf.InteractiveSession()
h = [1,0,0,0] # 단어를 원핫인코딩 : 수치화
e = [0,1,0,0]
l = [0,0,1,0]
o = [0,0,0,1]

# 변수공유를 위해 지정
with tf.variable_scope('five_sequences') as scope:
    hidden_size = 2
    # 4 -> 2 개의 특징으로 vector화된다.
    cell = tf.contrib.rnn.BasicRNNCell(num_units=hidden_size)
    x_data = np.array([[h,e,l,l,o]], dtype=np.float32)
    print(x_data.shape) # 1x5x4  : 데이터 함개 => 3차원으로
    pp.pprint(x_data)
    # cell의 개수 : 5개
    # cell당 입력데이터 : 4개
    # cell당 가중치 : 4x2
    outputs, states = tf.nn.dynamic_rnn(cell, x_data,dtype =tf.float32)
    sess.run(tf.global_variables_initializer())
    pp.pprint(outputs.eval())
    print('\n',outputs.shape,'\n') # 1x5x2
    print(states.shape,'\n') # 1x2
    pp.pprint(states.eval())

# 단어예측

In [64]:
tf.reset_default_graph()
char_arr = ['a','b','c','d','e',',f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z']
num_dic = {n: i for i, n in enumerate(char_arr)}
# a:0 번호를 키로
dic_len = len(num_dic)
# rule base => 자동툴 => 가중치만 이용하면 
seq_data = ['word', 'wood','deep','dive','cold','cool','load','love','kiss','kind']
# 10x3
# 

In [65]:
# 26x26행렬
# 1000000000000000000000000
# 0100000000000000000000000
# 0010000000000000000000000 ...
def make_batch(seq_data): # 자동으로 문자를 원핫 인코딩
    input_batch = []
    target_batch=[]
    for seq in seq_data: # word
        input = [num_dic[n] for n in seq[:-1]] # w=22, o=14, r=17
        target = num_dic[seq[-1]] # d=3
        input_batch.append(np.eye(dic_len)[input]) # 원핫 인코딩
        target_batch.append(target)
    return input_batch, target_batch

In [66]:
learning_rate = 0.01
n_hidden = 128
total_epoch = 30
n_step=3
n_input = n_class=dic_len # 26개

X = tf.placeholder(tf.float32, [None, n_step, n_input]) # 10x3x26
Y = tf.placeholder(tf.int32, [None]) # 10x26
# FFNN
W = tf.Variable(tf.random_normal([n_hidden, n_class])) # 128x26
b = tf.Variable(tf.random_normal([n_class])) # 26

In [67]:
cell1 = tf.nn.rnn_cell.BasicLSTMCell(n_hidden) # 특성수 128
# 계산한 양이 많아서 50%만 남기고 계산
# 과적합 방지 -> dropout
cell1 = tf.nn.rnn_cell.DropoutWrapper(cell1, output_keep_prob=0.5,seed=10)
cell2 = tf.nn.rnn_cell.BasicLSTMCell(n_hidden)
# 2레이어로 구성된 멀티셀
multi_cell = tf.nn.rnn_cell.MultiRNNCell([cell1, cell2])
outputs, states = tf.nn.dynamic_rnn(multi_cell, X,dtype=tf.float32)

# outputs : 10x3x128
# states : 10x128

In [68]:
outputs = tf.transpose(outputs, [1,0,2]) # 3x10x128 
outputs = outputs[-1] # 맨뒤에쓸 셀 10x1218
model = tf.matmul(outputs, W)+b # 비선형 회귀방식
# 분류
cost = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=model, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

In [69]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
input_batch , target_batch = make_batch(seq_data) # 단어데이터

for epoch in range(total_epoch):
    _, loss = sess.run([optimizer, cost], feed_dict={X:input_batch, Y:target_batch})
    print('Epoch : %04d'%(epoch+1), 'cost = {:.6f}'.format(loss))

Epoch : 0001 cost = 3.554584
Epoch : 0002 cost = 2.609149
Epoch : 0003 cost = 1.441095
Epoch : 0004 cost = 1.579632
Epoch : 0005 cost = 0.680771
Epoch : 0006 cost = 0.897215
Epoch : 0007 cost = 0.593994
Epoch : 0008 cost = 0.424144
Epoch : 0009 cost = 0.325358
Epoch : 0010 cost = 0.234904
Epoch : 0011 cost = 0.199243
Epoch : 0012 cost = 0.304904
Epoch : 0013 cost = 0.255864
Epoch : 0014 cost = 0.165813
Epoch : 0015 cost = 0.082868
Epoch : 0016 cost = 0.115473
Epoch : 0017 cost = 0.116674
Epoch : 0018 cost = 0.103213
Epoch : 0019 cost = 0.063181
Epoch : 0020 cost = 0.080963
Epoch : 0021 cost = 0.104196
Epoch : 0022 cost = 0.016710
Epoch : 0023 cost = 0.053694
Epoch : 0024 cost = 0.044977
Epoch : 0025 cost = 0.007708
Epoch : 0026 cost = 0.002861
Epoch : 0027 cost = 0.102131
Epoch : 0028 cost = 0.036725
Epoch : 0029 cost = 0.020556
Epoch : 0030 cost = 0.001445


In [72]:
prediction = tf.cast(tf.argmax(model,1), tf.int32)
prediction_check = tf.equal(prediction, Y)
accuracy = tf.reduce_mean(tf.cast(prediction_check, tf.float32))

input_batch, target_batch = make_batch(seq_data)
predict, accuracy_val = sess.run([prediction, accuracy], feed_dict={X:input_batch, Y:target_batch})

In [ ]:
# 문제 predict한 결과출력
# accuracy 출력
# 입력단어와 예측단어 결과를 나란히 출력

In [82]:

print(predict)
print(accuracy_val)

[ 3  3 15  4  3 11  3  4 18  3]
1.0


In [87]:
predict_words = []
for idx, val in enumerate(seq_data):
    last_char = char_arr[predict[idx]]
    predict_words.append(val[:3] + last_char)
print('입력값 : ',[w[:3] + ' ' for w in seq_data])
print('예측값 : ',predict_words)
print('정확도 : ',accuracy_val)

입력값 :  ['wor ', 'woo ', 'dee ', 'div ', 'col ', 'coo ', 'loa ', 'lov ', 'kis ', 'kin ']
예측값 :  ['word', 'wood', 'deep', 'dive', 'cold', 'cool', 'load', 'love', 'kiss', 'kind']
정확도 :  1.0
